In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,Dropout
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams.update({'font.size':20})
plt.rcParams["figure.figsize"] = (12,10)

In [2]:
str_files = ''
#Carrega os dados do arquivo .csv
ugrhi06 = pd.read_csv(str_files + "cetesb_concatenado06semoutliers.csv",encoding='utf-8',sep=',',index_col=[0])

#Carrega os dados do arquivo .csv
ugrhi10 = pd.read_csv(str_files + "cetesb_concatenado10semoutliers.csv",encoding='utf-8',sep=',',index_col=[0])
ugrhi10

,solido,ph,coliformes,dbo,fosforo,od,temperatura,turbidez
0,2.280559,0.908166,3.548615,0.772077,0.318589,0.540538,1.357776,1.456998
1,2.286276,0.908804,3.535570,0.761761,0.318589,0.536580,1.361072,1.461206
2,2.291917,0.908804,3.522122,0.751194,0.318589,0.532586,1.364343,1.465373
3,2.297487,0.909592,3.508243,0.740363,0.318589,0.528555,1.367589,1.469500
4,2.302986,0.909592,3.493907,0.729255,0.318589,0.524486,1.370811,1.473588
...,...,...,...,...,...,...,...,...
497,2.559008,0.910426,3.755951,0.929419,0.078276,0.681241,1.414973,1.477121
498,2.539782,0.910426,3.755951,0.983401,0.077413,0.685742,1.419129,1.540955
499,2.519664,0.914211,3.755951,1.031408,0.076549,0.690196,1.423246,1.596597
500,2.498569,0.910291,3.755951,1.074634,0.075684,0.694605,1.427324,1.645913


In [3]:
print('---- Tamanho dos conjuntos originais ----')
print('X:',len(ugrhi06))
print('y:',len(ugrhi10))

#Seleciona os dados equivalente ao tamanho da URGHI10
ugrhi06 = ugrhi06.iloc[0:len(ugrhi10)]

print('---- Conjunto de dados y igualado ao tamanho de dados do conjunto X ----')
print('X:',len(ugrhi06))
print('y:',len(ugrhi10))

def prepara_x_y_train_previsao(df06,df10,p):
    #Seleciona os dados 
    X = df06.iloc[:]
    #Remove a coluna ph, pois os seus valores serão preditos na UGRHI10
    X = X.drop([p],axis = 1).values
    #Seleciona os valores da coluna ph da URGHI06, na qual será utilizado para a normalização dos dados quando apresentar o gráfico
    train_previsao = df06[p].values
    #Seleciona os valores da coluna ph da URGHI10.
    y = df10[p].values
    
    X = X.reshape(len(X),7)
    y = y.reshape(len(y),1)
    train_previsao = train_previsao.reshape(len(train_previsao),1)
    
    return X,y,train_previsao


#assert X.shape == (len(X),7)
#assert y.shape == (len(y),1)

#print('---- Conjunto de dados y igualado ao tamanho de dados do conjunto X ----')
#print('X:',len(X))
#print('y:',len(y))


---- Tamanho dos conjuntos originais ----
X: 503
y: 502
---- Conjunto de dados y igualado ao tamanho de dados do conjunto X ----
X: 502
y: 502


In [4]:
#Funções para cálculos das métricas de desempenho e ajuste da estrutura dos dados.
'''
def mape(y_pred,y_true):
    mape_sum = 0
    for real,prediction in zip(y_true,y_pred):
        mape_sum += (abs((real - prediction))/real)
        mape = mape_sum/len(real)
    return mape
'''

def mape(y_pred,y_true):
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    return mape

def rmse(y_pred,y_true):
    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    return rmse

def r2(y_pred,y_true):
    rscore = r2_score(y_true,y_pred)
    return rscore

#Função para calcular o coeficiente de determinação entre os valores observados e preditos.
#Calcula-se a correlação entre os valores observados e preditos e eleva ao expoente 2.
def correlacao_determinacao(dtframe,tipo):
    
    if (tipo == 0): #Treino
        resultado = dtframe.corr().previsao_treino.values[1] ** 2 
        
    else: #teste
        resultado = dtframe.corr().previsao_teste.values[1] ** 2
            
    return resultado

def pegar_dados_coluna_test(trainingd,percent,index_coluna):
    data = trainingd.iloc[:,:].values
    train_previsao = trainingd.iloc[0:int(len(data)*percent),index_coluna:index_coluna+1].values
           
    return train_previsao

def ajusta_lista(array):
    lista = []
    
    for i in range(len(array)):
        lista.append(array[i][0])
            
    return lista

#Monta o Dataframe para calcular a correlação e coeficiente de determinação entre os valores observados e preditos.
def df_correlacao(previsoes_treino,treino,previsoes,observado_test):
    
    #print(ajusta_lista(treino))
    
    df_corr_determinacao_treino = pd.DataFrame()
    df_corr_determinacao_teste  = pd.DataFrame()
      
    #Calcula o coeficiente de determinação
    
    dict = {'previsao_treino': ajusta_lista(previsoes_treino), 'treino': ajusta_lista(treino)} 
        
    df_treino = pd.DataFrame(dict)
    dframes_treino = [df_corr_determinacao_treino,df_treino]
    df_corr_determinacao_treino = pd.concat(dframes_treino)
    
    dict = {'previsao_teste': ajusta_lista(previsoes), 'teste': ajusta_lista(observado_test)} 
    
    df_teste = pd.DataFrame(dict)
    dframes_teste = [df_corr_determinacao_teste,df_teste]
    df_corr_determinacao_teste = pd.concat(dframes_teste)
    
    r_treino = round(correlacao_determinacao(df_corr_determinacao_treino,0),4)
    r_teste  = round(correlacao_determinacao(df_corr_determinacao_teste,1),4)
                
    return r_treino,r_teste

In [5]:
df_resultado_medio_g = pd.DataFrame()

#index_coluna = 0

parametro = []

#média
media_parametro_mape_treino = []
media_parametro_rmse_treino = []
media_parametro_r_treino    = []
   
media_parametro_mape_teste  = []
media_parametro_rmse_teste  = []
media_parametro_r_teste     = []
   
#desvio_padrão
   
#std_parametro_mape_treino = []
std_parametro_rmse_treino = []
#std_parametro_r_treino    = []
   
#std_parametro_mape_teste  = []
std_parametro_rmse_teste  = []
#std_parametro_r_teste     = []

tecnica = []

for p in ('ph','coliformes', 'dbo','fosforo','od','solido','temperatura','turbidez'):
   
    print('Agora é a vez do parâmetro:', p)
    
    X,y,train_previsao = prepara_x_y_train_previsao(ugrhi06,ugrhi10,p)

    #Normalização dos dados: Normaliza os dados dentro um intervalo (0 a 1).
    sc = MinMaxScaler()
    X = sc.fit_transform(X)
    y = sc.fit_transform(y)

    #Normalização dos dados de previsão para teste
    normalizador_previsao = MinMaxScaler()
    normalizador_previsao.fit_transform(train_previsao)

    #Separação dos dados para treinamento(70%) e teste(30%)
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)

    #am = ['rl','rf','mlp','lstm']
    
    media_simulador_mape_treino = []
    media_simulador_rmse_treino = []
    media_simulador_r_treino    = []
            
    media_simulador_mape_teste  = []
    media_simulador_rmse_teste  = []
    media_simulador_r_teste     = []

    for t in ('rl','rf','mlp','lstm'):
        if t == 'rl':
            #print('REGRESSÃO LINEAR')
            model = LinearRegression()
        
            #Treina o modelo
            model.fit(X_train,y_train)
        
            #Predição dos dados de teste
            y_pred = model.predict(X_test)

            y_pred_rl = sc.inverse_transform(y_pred)

            X_test_rl = sc.inverse_transform(X_test)
            y_test_rl = sc.inverse_transform(y_test)
        
            #Predição dos dados de treinamento
            y_predt = model.predict(X_train)
            y_predt_rl = sc.inverse_transform(y_predt)
        
            y_train_rl = sc.inverse_transform(y_train)
        
            '''
            plt.plot(y_test_rl,color='red', label='pH Observado')
            plt.plot(y_pred_rl,color='blue', label='pH predito')
            plt.xlabel('Semanas')
            plt.ylabel('Valor de pH')
            plt.legend()
            plt.title('Predição Regressão Linear')
            plt.show()
            '''
        
            mape_treino_rl       = round(mape(y_predt_rl,y_train_rl),2)
            rmse_treino_rl       = round(rmse(y_predt_rl,y_train_rl),2)
                
            mape_teste_rl        = round(mape(y_pred_rl,y_test_rl),2)
            rmse_teste_rl        = round(rmse(y_pred_rl,y_test_rl),2)
                
            r_treino_rl,r_teste_rl = df_correlacao(y_predt_rl,y_train_rl,y_pred_rl,y_test_rl)
            
            media_simulador_mape_treino.append(np.mean(mape_treino_rl))
            media_simulador_rmse_treino.append(np.mean(rmse_treino_rl))
            media_simulador_r_treino.append(np.mean(r_treino_rl))
            
            media_simulador_mape_teste.append(np.mean(mape_teste_rl))
            media_simulador_rmse_teste.append(np.mean(rmse_teste_rl))
            media_simulador_r_teste.append(np.mean(r_teste_rl))
            
            tecnica.append('RL (Todas var.)')
            parametro.append(p)
                        
        elif t == 'rf':
            #print('RANDON FOREST')
        
            model = RandomForestRegressor()
        
            #Treina o modelo
            model.fit(X_train,y_train)

            #Predição dados de teste
            y_pred = model.predict(X_test)
                
            y_pred_rf = sc.inverse_transform(y_pred.reshape(-1, 1))
        
            X_test_rf = sc.inverse_transform(X_test)
            y_test_rf = sc.inverse_transform(y_test)
        
            #Predição dados de treinamento
        
            y_predt = model.predict(X_train)
            y_predt_rf = sc.inverse_transform(y_predt.reshape(-1, 1))
        
            y_train_rf = sc.inverse_transform(y_train)
        
            '''
            plt.plot(y_test_rf,color='red', label='pH Observado')
            plt.plot(y_pred_rf,color='blue', label='pH predito')
            plt.xlabel('Semanas')
            plt.ylabel('Valor de pH')
            plt.legend()
            plt.title('Predição Random Forest')
            plt.show()
            '''
        
            mape_treino_rf       = round(mape(y_predt_rf,y_train_rf),2)
            rmse_treino_rf       = round(rmse(y_predt_rf,y_train_rf),2)
        
            mape_teste_rf        = round(mape(y_pred_rf,y_test_rf),2)
            rmse_teste_rf        = round(rmse(y_pred_rf,y_test_rf),2)
                
            r_treino_rf,r_teste_rf = df_correlacao(y_predt_rf,y_train_rf,y_pred_rf,y_test_rf)
            
            media_simulador_mape_treino.append(np.mean(mape_treino_rf))
            media_simulador_rmse_treino.append(np.mean(rmse_treino_rf))
            media_simulador_r_treino.append(np.mean(r_treino_rf))
            
            media_simulador_mape_teste.append(np.mean(mape_teste_rf))
            media_simulador_rmse_teste.append(np.mean(rmse_teste_rf))
            media_simulador_r_teste.append(np.mean(r_teste_rf))
            
            tecnica.append('RF (Todas var.)')
            parametro.append(p)
        
        
        elif t == 'mlp':
        
            #print('MLP')           
            
        
            for r in range(0,6):
                               
                model = Sequential()
                model.add(Dense(units = 10, activation = 'relu'))
                model.add(Dense(units = 21, activation = 'relu'))
                model.add(Dense(units = 1, activation = 'sigmoid'))
                model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])
        
                es = EarlyStopping(monitor='val_loss', patience = 3, verbose=0)


                #Treina o modelo
                history = model.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size = 32, epochs = 2000, 
                        callbacks=[es], verbose=0)

                #Predição com os dados de test
                y_pred = model.predict(X_test)
                y_pred_mlp = normalizador_previsao.inverse_transform(y_pred)

                X_test_mlp = sc.inverse_transform(X_test)
                y_test_mlp = sc.inverse_transform(y_test)
        
                #Predição com os dados de treinamento
                y_predt = model.predict(X_train)
                y_predt_mlp = normalizador_previsao.inverse_transform(y_predt)
        
                y_train_mlp = sc.inverse_transform(y_train)
        
                '''
                plt.plot(y_test_mlp,color='red', label='pH Observado')
                plt.plot(y_pred_mlp,color='blue', label='pH predito')
                plt.xlabel('Semanas')
                plt.ylabel('Valor de pH')
                plt.legend()
                plt.title('Predição MLP')
                plt.show()
                '''
        
                mape_treino_mlp_sigmoid       = round(mape(y_predt_mlp,y_train_mlp),2)
                rmse_treino_mlp_sigmoid       = round(rmse(y_predt_mlp,y_train_mlp),2)
        
                mape_teste_mlp_sigmoid        = round(mape(y_pred_mlp,y_test_mlp),2)
                rmse_teste_mlp_sigmoid        = round(rmse(y_pred_mlp,y_test_mlp),2)
        
                r_treino_mlp_sigmoid,r_teste_mlp_sigmoid = df_correlacao(y_predt_mlp,y_train_mlp,y_pred_mlp,y_test_mlp)
               
                media_simulador_mape_treino.append(np.mean(mape_treino_mlp_sigmoid))
                media_simulador_rmse_treino.append(np.mean(rmse_treino_mlp_sigmoid))
                media_simulador_r_treino.append(np.mean(r_treino_mlp_sigmoid))
            
                media_simulador_mape_teste.append(np.mean(mape_teste_mlp_sigmoid))
                media_simulador_rmse_teste.append(np.mean(rmse_teste_mlp_sigmoid))
                media_simulador_r_teste.append(np.mean(r_teste_mlp_sigmoid))
                
                
                '''
                # plot training history
                plt.plot(history.history['loss'], label='train')
                plt.plot(history.history['val_loss'], label='test')
                plt.legend()
                plt.show()
                '''
                
            tecnica.append('MLP (Todas var.)')
            parametro.append(p)
                   
    
        else:
        
                #print('LSTM')
                    
            X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
                
                                
            for r in range(0,6):
                
                model = Sequential()
                model.add(LSTM(units = 10, input_shape = (X_train.shape[1], 1)))
                model.add(Dense(21, activation = 'relu'))
                model.add(Dropout(0.2))
                model.add(Dense(1, activation = 'sigmoid'))
                model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

                es = EarlyStopping(monitor='val_loss', patience = 3, verbose=0)


                #Treina o modelo
                history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 32, epochs = 2000, 
                        callbacks=[es], verbose=0)

                #Predição com dados de teste
                y_pred = model.predict(X_test)
                y_pred_lstm = sc.inverse_transform(y_pred)

                y_test_lstm = sc.inverse_transform(y_test)
                y_test_lstm.shape
        
                #Predição com dados de treinamento
                y_predt = model.predict(X_train)
                y_predt_lstm = sc.inverse_transform(y_predt)

                y_train_lstm = sc.inverse_transform(y_train)
        
                '''
                plt.plot(y_test_lstm,color='red', label='pH Observado')
                plt.plot(y_pred_lstm,color='blue', label='pH predito')
                plt.xlabel('Semanas')
                plt.ylabel('Valor de pH')
                plt.legend()
                plt.title('Predição LSTM')
                plt.show()
        
                '''
        
                mape_treino_lstm_sigmoid       = round(mape(y_predt_lstm,y_train_lstm),2)
                rmse_treino_lstm_sigmoid       = round(rmse(y_predt_lstm,y_train_lstm),2)
                
                mape_teste_lstm_sigmoid        = round(mape(y_pred_lstm,y_test_lstm),2)
                rmse_teste_lstm_sigmoid        = round(rmse(y_pred_lstm,y_test_lstm),2)
                
                r_treino_lstm_sigmoid,r_teste_lstm_sigmoid = df_correlacao(y_predt_lstm,y_train_lstm,y_pred_lstm,y_test_lstm)
                
                '''
                # plot training history
                plt.plot(history.history['loss'], label='train')
                plt.plot(history.history['val_loss'], label='test')
                plt.legend()
                plt.show()
                '''
                    
                media_simulador_mape_treino.append(np.mean(mape_treino_lstm_sigmoid))
                media_simulador_rmse_treino.append(np.mean(rmse_treino_lstm_sigmoid))
                media_simulador_r_treino.append(np.mean(r_treino_lstm_sigmoid))
            
                media_simulador_mape_teste.append(np.mean(mape_teste_lstm_sigmoid))
                media_simulador_rmse_teste.append(np.mean(rmse_teste_lstm_sigmoid))
                media_simulador_r_teste.append(np.mean(r_teste_lstm_sigmoid))
                
                                    
            tecnica.append('LSTM (Todas var.)')
            parametro.append(p)
                    
                #média
        media_parametro_mape_treino.append(np.mean(media_simulador_mape_treino))
        media_parametro_rmse_treino.append(np.mean(media_simulador_rmse_treino))
        media_parametro_r_treino.append(np.mean(media_simulador_r_treino))
    
        media_parametro_mape_teste.append(np.mean(media_simulador_mape_teste))
        media_parametro_rmse_teste.append(np.mean(media_simulador_rmse_teste))
        media_parametro_r_teste.append(np.mean(media_simulador_r_teste))
        
        #desvio_padrão
    
        #std_parametro_mape_treino.append(np.std(media_lag_mape_treino))
        std_parametro_rmse_treino.append(np.std(media_simulador_rmse_treino))
        #std_parametro_r_treino.append(np.std(media_lag_r_treino))
    
        #std_parametro_mape_teste.append(np.std(media_lag_mape_teste))
        std_parametro_rmse_teste.append(np.std(media_simulador_rmse_teste))
        #std_parametro_r_teste.append(np.std(media_lag_r_teste))
'''
print(len(parametro))
print(len(tecnica))
print(len(media_parametro_mape_treino))
print(len(media_parametro_rmse_treino))
print(len(std_parametro_rmse_treino))
print(len(media_parametro_r_treino))
print(len(media_parametro_mape_teste))
print(len(media_parametro_rmse_teste))
print(len(std_parametro_rmse_teste))
print(len(media_parametro_r_teste))
'''    
    
dict = {'parametro': parametro,
        'AM':tecnica,
        'mape_treino':  media_parametro_mape_treino,
        'rmse_treino':  media_parametro_rmse_treino, 
        'std_rmse_treino':  std_parametro_rmse_treino,
        'r_quad_treino':media_parametro_r_treino,    
        'mape_teste':   media_parametro_mape_teste,   
        'rmse_teste':   media_parametro_rmse_teste,  
        'std_rmse_teste':   std_parametro_rmse_teste,
        'r_quad_teste': media_parametro_r_teste}
   
df_resultado_final = pd.DataFrame(dict)

dframes = [df_resultado_medio_g,df_resultado_final]
df_resultado_medio_g = pd.concat(dframes)
df_resultado_medio_g.round(2)


Agora é a vez do parâmetro: ph


/home/anderson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Agora é a vez do parâmetro: coliformes


/home/anderson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Agora é a vez do parâmetro: dbo


...

Agora é a vez do parâmetro: temperatura


/home/anderson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Agora é a vez do parâmetro: turbidez


/home/anderson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


,parametro,AM,mape_treino,rmse_treino,std_rmse_treino,r_quad_treino,mape_teste,rmse_teste,std_rmse_teste,r_quad_teste
0,ph,RL (Todas var.),0.16,0.00,0.00,0.12,0.15,0.00,0.00,0.08
1,ph,RF (Todas var.),0.11,0.00,0.00,0.53,0.16,0.00,0.00,0.06
2,ph,MLP (Todas var.),2.15,0.02,0.01,0.20,2.19,0.02,0.01,0.05
3,ph,LSTM (Todas var.),1.30,0.01,0.01,0.13,1.32,0.01,0.01,0.06
4,coliformes,RL (Todas var.),46.31,1.05,0.00,0.08,53.39,1.09,0.00,0.11
5,coliformes,RF (Todas var.),31.79,0.73,0.32,0.51,51.90,1.08,0.01,0.12
6,coliformes,MLP (Todas var.),73.62,1.69,0.58,0.16,87.78,1.84,0.44,0.07
7,coliformes,LSTM (Todas var.),64.09,1.46,0.51,0.11,76.24,1.57,0.45,0.06
8,dbo,RL (Todas var.),18.18,0.22,0.00,0.25,17.29,0.22,0.00,0.18
9,dbo,RF (Todas var.),12.37,0.16,0.06,0.59,16.36,0.22,0.01,0.22


In [6]:
df_resultado_medio_g.round(2).to_csv('/home/anderson/Downloads/src_experimentos_dissertação/resultados_log_csv/espacial_todas_var.csv',index=True,header=True)